In [ ]:
# Databricks notebook source

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Initialize SparkSession if not already available
spark = SparkSession.builder.appName("Products Transform").getOrCreate()

# Define Unity Catalog catalog and schema names
# IMPORTANT: Replace 'your_catalog' and 'your_schema' with your actual desired Unity Catalog catalog and schema names.
catalog_name = "main_catalog"
schema_name = "silver_products_schema"

# Create catalog and schema if they don't exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
spark.sql(f"USE SCHEMA {schema_name}")

input_table_name = "main_catalog.bronze_products_schema.products_raw"

# Define the Unity Catalog output table name
output_table_name = f"{catalog_name}.{schema_name}.products_transformed"

# Read the raw data from the Unity Catalog table
df = spark.read.table(input_table_name)

# Perform transformations
df_transformed = df.withColumn("price", col("price").cast("double")) \
    .withColumn("brand", when(col("name").contains("Awesome"), "AwesomeBrand").otherwise("GenericBrand")) \
    .select("product_id", "name", "brand", "price", "created_at")

# Write the transformed data to a Unity Catalog Delta table
# Ensure you have appropriate permissions to write to the specified catalog and schema.
df_transformed.write.format("delta").mode("overwrite").saveAsTable(output_table_name)

print(f"Successfully transformed products data and saved to Unity Catalog table: {output_table_name}")
